In [2]:
import rasterio 
import rasterio.features 
import rasterio.warp 
import numpy as np 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 

In [ ]:
dataset = rasterio.open("./Code/proba_log_reg_l1_extract.tif")
dataset = dataset.read()
dataset2 = rasterio.open('./Code/class_log_reg_l1_extract.tif')
dataset2 = dataset2.read()

In [3]:
print(np.sum(dataset[:,321,89])) #On somme les probas d'appartenance du pixel (321,89) à toutes les classes, le résultat doit valoir 1 
print(np.sum(dataset[:,1,2])) 

1.0
0.0


In [4]:
print(dataset[:,321,89])

[3.0893169e-08 8.0709122e-02 6.7250209e-04 9.1566640e-01 7.8258058e-04
 1.6859564e-03 1.4423195e-08 4.8079420e-04 2.4016593e-13 2.6223506e-06]


In [5]:
print(np.shape(dataset))

(10, 624, 361)


In [29]:
#Creation d'une liste de 624*89 éléments contenant les vecteurs de probabilités d'appartenance d'un pixel à une classe
nb_class = np.shape(dataset)[0] 
nb_raws = np.shape(dataset)[1] 
nb_columns = np.shape(dataset)[2]
vect_nul = np.zeros(nb_class).tolist()
#Les listes ci-dessous contiennent les probabilités d'appartenance d'un pixel à chaque classe ainsi que ses coordonnées (ligne, colonne) en avant-dernier et dernier élément
liste_probas_fiables_sans_coord = [] #pixels qui vont pouvoir entrainer notre modèle
liste_probas_fiables_avec_coord = [] #même que celui d'au dessus, mais avec les coordonnées des points en plus
liste_probas_ombres = [] #pixels d'ombre
liste_probas_apredire = [] #pixels ambigus => kmeans nous permettra de les attribuer en fonction d'un certain seuil à la bonne classe

for i in range(nb_raws) :
    for j in range(nb_columns) :
        aux = dataset[:,i,j].tolist()
        #Repérage des pixels ayant été classés avec une probabilité inférieure à 0.5
        threshold = 0.5
        bool = all(proba < threshold for proba in aux) 
            #bool = False si au moins une probabilité est supérieure à 0.5
            #bool = True si aucune probabilité n'excède 0.5

        if aux == vect_nul : #On rejette du modèle les pixels d'ombre 
            liste_probas_ombres.append(dataset[:,i,j].tolist() + [i] + [j])  
        if not(bool) : #On rejette aussi ceux ayant été classés avec une probabilité inférieure à 0.5 et que l'on devra prédire à la fin 
            liste_probas_apredire.append(dataset[:,i,j].tolist() + [i] + [j])  
        else : 
            liste_probas_fiables_sans_coord.append(dataset[:,i,j].tolist() + [i] + [j])

In [31]:
print(len(liste_probas)) 
#127622 au lieu de 225264 ce qui fait environ 56% des données que l'on peut considérer fiables pour entraîner le modèle

127622


In [33]:
kmeans = KMeans(n_clusters=9, random_state=0).fit(liste_probas_fiables_sans_coord) #9 clusters et non 10 car on a supprimé les pixels d'ombre
labels = kmeans.labels_ 
print(labels)

[3 3 3 ... 8 5 8]


Remarque : pour ensuite garder le lien entre les pixels dans les listes on peut faire un dictionnaire pour sauvegarder ses coordonnées dans l'image

In [35]:
#Centre des clusters 
centers = kmeans.cluster_centers_
#print(centers)

In [36]:
#Distance de chaque point à leur cluster leur plus proche
distance = kmeans.inertia_

In [37]:
distance

5981.029368828313

In [43]:
essai = dataset[:,321,89].tolist()
print(essai)
liste1 = []
liste1.append(dataset[:,321,89].tolist() + [321] + [89])

[3.089316891191629e-08, 0.08070912212133408, 0.0006725020939484239, 0.915666401386261, 0.0007825805805623531, 0.0016859563766047359, 1.4423195082713391e-08, 0.0004807942023035139, 2.401659329308997e-13, 2.622350621095393e-06]
